# Missing values

<p>In this notebook, we will analyze the dataset for missing values, as handling missing data is a critical aspect of ensuring the <b>completeness</b>, one of the fundamentals of <b>Data Quality</b>, during the implementation of a machine learning project.</p>

<p>Given that the explanatory variables represent time series, we will treat the group of column series (e.g., col-*) as a single variable type and inspect them in detail.</p>

In [3]:
import pandas as pd
import os

base_dir = os.path.abspath(os.path.join('..', '..', '..', 'data', 'raw'))
file_path = os.path.join(base_dir, 'train.csv') 

patients = pd.read_csv(file_path, low_memory=False)
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177024 entries, 0 to 177023
Columns: 508 entries, id to bg+1:00
dtypes: float64(433), object(75)
memory usage: 686.1+ MB


### Percentage of missing values for bg-* columns

In [4]:
# select all the 'bg-*' columns except the target one
bg_cols = [col for col in patients.columns if col.startswith('bg-')]

# calculates the percentage of missing values for each bg-* column individually and gives the min and max percentage across the columns
missing_percentage = patients[bg_cols].isnull().mean() * 100
print(min(missing_percentage), max(missing_percentage))

1.5229573391178597 15.399606832971802


In [5]:
missing_percentage = patients[bg_cols].isna().sum().sum() / patients[bg_cols].size * 100
missing_percentage

np.float64(10.761911755844782)

<b>Conclusion & handling:</b> There are between 1.5% and 15% missing values across the bg-* columns. Given that these data are recorded by a continuous glucose monitor (CGM), with some patients having data recorded at 15-minute intervals, we assume that after proper interpolation and imputation, the missing values will be resolved. This step will be carried out during the data preprocessing phase.

### Percentage of missing values for insulin-* columns

In [7]:
insulin_cols = [col for col in patients.columns if col.startswith('insulin-')]

# Calculate the percentage of missing values for each 'insulin-*' column individually and gives the min and max percentage across the columns
missing_percentage = patients[insulin_cols].isnull().mean() * 100
print(min(missing_percentage), max(missing_percentage))

5.295892082429502 5.335999638467101


<b>Conclusion & handling:</b> There are approximately 5.3% missing values accross the insulin-* columns. Given that these data are recorded by the insulin pump, with some patients having data recorded at 15-minute intervals, we assume that after proper interpolation and imputation, the missing values will be resolved. This step will be carried out during the data preprocessing phase.

### Percentage of missing values for carbs-* columns

In [9]:
carbs_cols = [col for col in patients.columns if col.startswith('carbs-')]

# Calculate the percentage of missing values for each 'carbs-* column individually and gives the min and max percentage across the columns
missing_percentage = patients[carbs_cols].isnull().mean() * 100
print(min(missing_percentage), max(missing_percentage))

98.53918112798264 98.57194504699928


<b>Conclusion & handling:</b>  There are approximately 98.5% missing values accross the hr-* columns. Since these data are self-reported by participants and may not be reliable, we assume this variable will not significantly impact future predictions and can be dropped from the model.

### Percentage of missing values for hr-* columns

In [10]:
hr_cols = [col for col in patients.columns if col.startswith('hr-')]

# Calculate the percentage of missing values for each 'hr-*' column individually and gives the min and max percentage across the columns
missing_percentage = patients[hr_cols].isnull().mean() * 100
print(min(missing_percentage), max(missing_percentage))

28.885348879248014 29.272302060737527


In [11]:
patients[hr_cols].isna().sum().sum() / patients[hr_cols].size * 100

np.float64(29.10068377420262)

<b>Conclusion & handling:</b> There are approximately 29.1% missing values accross the hr-* columns. Given that these data are recorded by the smartwatch, with some patients having data recorded at 15-minute intervals, we assume that after proper interpolation and imputation, the missing values will be partially resolved. Any remaining missing values will be imputed using an appropriate method (to be determined). This step will be carried out during the data preprocessing phase.

### Percentage of missing values for steps-* columns

In [12]:
steps_cols = [col for col in patients.columns if col.startswith('steps-')]

# Calculate the percentage of missing values for each 'steps-* column individually and gives the min and max percentage across the columns
missing_percentage = patients[steps_cols].isnull().mean() * 100
print(min(missing_percentage), max(missing_percentage))

53.66165039768619 54.055382321041215


In [13]:
patients[steps_cols].isna().sum().sum() / patients[steps_cols].size * 100

np.float64(53.87635763135695)

<b>Conclusion & handling:</b> There are approximately 54% missing values accross the steps-* columns. Given that these data are recorded by the smartwatch, with some patients having data recorded at 15-minute intervals, we assume that after proper interpolation and imputation, the missing values will be partially resolved. Any remaining missing values will be replaced with 0 (to be confirmed). This step will be carried out during the data preprocessing phase.

### Percentage of missing values for cals-* columns

In [14]:
cals_cols = [col for col in patients.columns if col.startswith('cals-')]

# Calculate the percentage of missing values for each 'cals-*' column individually and gives the min and max percentage across the columns
missing_percentage = patients[cals_cols].isnull().mean() * 100
print(min(missing_percentage), max(missing_percentage))

19.916508496023138 20.231155097613883


In [15]:
patients[cals_cols].isna().sum().sum() / patients[cals_cols].size * 100

np.float64(20.07893939051579)

<b>Conclusion & handling:</b> There are approximately 20% missing values accross the cals-* columns. Given that these data are recorded by the smartwatch, with some patients having data recorded at 15-minute intervals, we assume that after proper interpolation and imputation, the missing values will be partially resolved. Any remaining missing values will be replaced with 0. This step will be carried out during the data preprocessing phase.

### Percentage of missing values for activity-* columns

In [17]:
activity_cols = [col for col in patients.columns if col.startswith('activity-')]
# Calculate the percentage of missing values for each 'activity-*' column individually and gives the min and max percentage across the columns
missing_percentage = patients[activity_cols].isnull().mean() * 100
print(min(missing_percentage), max(missing_percentage))

# Initialize an empty set to store unique activities
unique_activities = set()
for col in activity_cols:
    unique_activities.update(patients[col].dropna().unique())  
unique_activities_list = list(unique_activities)
print(unique_activities_list)

98.43411062906723 98.46800433839479
['Walking', 'Strength training', 'Weights', 'Dancing', 'Sport', 'Zumba', 'Hike', 'Spinning', 'HIIT', 'Aerobic Workout', 'Run', 'Swim', 'Yoga', 'Indoor climbing', 'Workout', 'Bike', 'Outdoor Bike', 'Tennis', 'Running', 'Stairclimber', 'Swimming', 'Walk']


##### Percentage of reported activities for each patient

In [18]:
# find the "busiest" patient, i.e. the one who reported the most activities across the activity-* columns
# Count non-missing values in the 'activity-*' columns for each patient
activity_counts = patients.groupby('p_num')[activity_cols].apply(lambda x: x.notnull().sum().sum())

# Find the patient with the maximum activity reports
busiest_patient = activity_counts.idxmax()
max_activities_reported = activity_counts.max()

# Output the result
print(f"The busiest patient is {busiest_patient} with {max_activities_reported} reported activities.")

# Note: this is the highest absolute number of activity entries 

The busiest patient is p10 with 65952 reported activities.


In [19]:
# Count non-missing values in the 'activity-*' columns for each patient
activity_counts = patients.groupby('p_num')[activity_cols].apply(lambda x: x.notnull().sum().sum())

# Calculate the total possible activity entries for each patient
total_possible_activities = len(activity_cols) * patients.groupby('p_num').size()

# Calculate the percentage of filled activity data for each patient
activity_percentage_filled = (activity_counts / total_possible_activities) * 100

# Display the percentage for each patient
print(activity_percentage_filled)

# Note: this is the highest percentage of filled activity data, which suggests that a higher proportion of the total data available 
# for this patient is filled in, even if the absolute number of activity records is lower.


p_num
p01    3.935650
p02    0.935857
p03    0.430306
p04    1.124625
p05    1.233879
p06    2.589401
p10    3.598649
p11    1.884432
p12    0.304525
dtype: float64


<b>Conclusion & handling:</b> There are approximately 98.4% missing values across the activity-* columns. Although these data are self-reported by participants and may not be entirely reliable, we assume this variable might still be useful for model predictions. For at least two participants who actively reported their activities, it would be interesting to evaluate whether the model shows significant improvements. If no substantial benefit is observed, this feature can be dropped. Further analysis on this is required.

### Percentage of missing values for the target variable bg+1:00

In [20]:
patients['bg+1:00'].isna().sum() / len(patients) * 100

np.float64(0.0)

<b>Conclusion & handling:</b> There are no missing values in the bg+1:00 column, so no further action is required.